# B2B Marketing Analytics: Hands-On Exercises

This notebook accompanies the B2B Marketing Guide Series. You'll work with a realistic synthetic dataset to practice:

- Data cleaning and preparation
- Conversion rate analysis
- Attribution modeling
- Cohort analysis
- Pipeline reporting

**Prerequisites:** Familiarity with concepts from Guides 1-3 (B2B fundamentals, martech stack, attribution models).

---
## Setup and Data Generation

First, we generate a realistic synthetic B2B dataset. The data reflects typical B2B patterns:
- 90-day average sales cycle
- ~20% MQL→SQL conversion
- ~25% SQL→Opportunity conversion  
- ~30% Opportunity→Closed Won
- Intentional data quality issues (duplicates, missing values, inconsistent formatting)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
random.seed(42)

In [ ]:
# Configuration
NUM_LEADS = 5000
NUM_ACCOUNTS = 800
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2024, 6, 30)

# Realistic company data
COMPANY_PREFIXES = ['Tech', 'Data', 'Cloud', 'Cyber', 'Smart', 'Digital', 'Global', 'Prime', 'Next', 'Core',
                    'Alpha', 'Beta', 'Apex', 'Peak', 'Vertex', 'Nova', 'Quantum', 'Fusion', 'Synergy', 'Nexus']
COMPANY_SUFFIXES = ['Solutions', 'Systems', 'Technologies', 'Labs', 'Analytics', 'Software', 'Dynamics',
                    'Innovations', 'Ventures', 'Partners', 'Group', 'Inc', 'Corp', 'Co', 'Industries']
INDUSTRIES = ['Technology', 'Financial Services', 'Healthcare', 'Manufacturing', 'Retail', 
              'Professional Services', 'Education', 'Media', 'Telecommunications', 'Energy']

LEAD_SOURCES = ['paid_search', 'linkedin', 'organic', 'referral', 'event', 'content_syndication']
LEAD_SOURCE_WEIGHTS = [0.25, 0.20, 0.15, 0.10, 0.15, 0.15]

# Self-reported sources (messy, realistic free-text responses)
SELF_REPORTED_TEMPLATES = {
    'paid_search': ['Google search', 'google', 'searched online', 'Google', 'web search', 'Googled it', 
                    'found you on google', 'internet search', None],
    'linkedin': ['LinkedIn', 'linkedin', 'LinkedIn ad', 'saw your post on LinkedIn', 'LI', 
                 'linkedin article', 'a linkedin post', None],
    'organic': ['blog post', 'your blog', 'article I read', 'content on your site', 'organic search',
                'found your website', 'SEO', None, None],  # More Nones = more missing
    'referral': ['colleague recommendation', 'friend told me', 'word of mouth', 'my boss mentioned you',
                 'referral from a friend', 'heard about you from someone', 'a peer recommended you', None],
    'event': ['conference', 'webinar', 'your webinar last month', 'trade show', 'SaaStr', 'Dreamforce',
              'met at an event', 'virtual event', None],
    'content_syndication': ['downloaded a whitepaper', 'ebook', 'read your report', 'G2', 'TrustRadius',
                            'content download', None, None, None]  # Often no attribution
}

# Dark funnel additions - sometimes people report sources that don't match tracking
DARK_FUNNEL_SOURCES = ['podcast', 'heard on a podcast', 'YouTube video', 'Twitter', 'community slack',
                       'newsletter', 'been following you for a while', "can't remember", 'not sure']

In [ ]:
def generate_company_name():
    """Generate realistic company names with some duplicates and variations."""
    prefix = random.choice(COMPANY_PREFIXES)
    suffix = random.choice(COMPANY_SUFFIXES)
    return f"{prefix} {suffix}"

def generate_email(name, company):
    """Generate email with realistic variations."""
    company_domain = company.lower().replace(' ', '').replace(',', '')[:12]
    domains = [f"{company_domain}.com", f"{company_domain}.io", f"{company_domain}.co"]
    name_parts = name.lower().split()
    
    formats = [
        f"{name_parts[0]}.{name_parts[1]}@{random.choice(domains)}",
        f"{name_parts[0][0]}{name_parts[1]}@{random.choice(domains)}",
        f"{name_parts[0]}@{random.choice(domains)}"
    ]
    return random.choice(formats)

def random_date(start, end):
    """Generate random date between start and end."""
    delta = end - start
    random_days = random.randint(0, delta.days)
    return start + timedelta(days=random_days)

In [ ]:
# Generate Accounts
accounts = []
company_names_generated = set()

for i in range(NUM_ACCOUNTS):
    company_name = generate_company_name()
    # Allow some duplicate names (realistic)
    if random.random() < 0.95:
        while company_name in company_names_generated:
            company_name = generate_company_name()
    company_names_generated.add(company_name)
    
    employee_count = int(np.random.lognormal(6, 1.5))  # Skewed toward smaller companies
    employee_count = min(max(employee_count, 10), 50000)
    
    # Revenue correlates with employees
    revenue_per_employee = random.uniform(80000, 300000)
    annual_revenue = int(employee_count * revenue_per_employee)
    
    # Tier based on company size
    if employee_count >= 1000 or annual_revenue >= 100_000_000:
        tier = 1
    elif employee_count >= 200 or annual_revenue >= 20_000_000:
        tier = 2
    else:
        tier = 3
    
    accounts.append({
        'account_id': f'ACC-{i+1:04d}',
        'company_name': company_name,
        'industry': random.choice(INDUSTRIES),
        'employee_count': employee_count,
        'annual_revenue': annual_revenue,
        'tier': tier
    })

accounts_df = pd.DataFrame(accounts)
print(f"Generated {len(accounts_df)} accounts")
print(f"Tier distribution: {accounts_df['tier'].value_counts().to_dict()}")

In [ ]:
# Generate Leads
FIRST_NAMES = ['James', 'Mary', 'John', 'Patricia', 'Robert', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth',
               'David', 'Barbara', 'Richard', 'Susan', 'Joseph', 'Jessica', 'Thomas', 'Sarah', 'Christopher', 'Karen',
               'Daniel', 'Lisa', 'Matthew', 'Nancy', 'Anthony', 'Betty', 'Mark', 'Margaret', 'Donald', 'Sandra',
               'Steven', 'Ashley', 'Paul', 'Kimberly', 'Andrew', 'Emily', 'Joshua', 'Donna', 'Kenneth', 'Michelle']
LAST_NAMES = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis', 'Rodriguez', 'Martinez',
              'Hernandez', 'Lopez', 'Gonzalez', 'Wilson', 'Anderson', 'Thomas', 'Taylor', 'Moore', 'Jackson', 'Martin',
              'Lee', 'Perez', 'Thompson', 'White', 'Harris', 'Sanchez', 'Clark', 'Ramirez', 'Lewis', 'Robinson']

leads = []
contact_to_account = []

for i in range(NUM_LEADS):
    # Assign to account (some leads, especially from larger accounts, share accounts)
    account = accounts_df.sample(1, weights=(accounts_df['employee_count'] ** 0.3)).iloc[0]
    
    first_name = random.choice(FIRST_NAMES)
    last_name = random.choice(LAST_NAMES)
    full_name = f"{first_name} {last_name}"
    
    # Company name variations (data quality issue)
    company_name_variations = [
        account['company_name'],
        account['company_name'].upper(),
        account['company_name'].lower(),
        account['company_name'].replace(' ', ''),
        account['company_name'] + ', Inc.',
        account['company_name'] + ' LLC'
    ]
    company_name = random.choices(
        company_name_variations, 
        weights=[0.7, 0.05, 0.05, 0.05, 0.1, 0.05]
    )[0]
    
    lead_source = random.choices(LEAD_SOURCES, weights=LEAD_SOURCE_WEIGHTS)[0]
    
    # Self-reported source (messy, with dark funnel)
    if random.random() < 0.15:  # 15% dark funnel
        self_reported = random.choice(DARK_FUNNEL_SOURCES)
    else:
        self_reported = random.choice(SELF_REPORTED_TEMPLATES[lead_source])
    
    created_date = random_date(START_DATE, END_DATE)
    
    # Lead score (higher for bigger companies, certain sources)
    base_score = random.randint(20, 60)
    if account['tier'] == 1:
        base_score += 25
    elif account['tier'] == 2:
        base_score += 10
    if lead_source in ['referral', 'event']:
        base_score += 15
    lead_score = min(base_score + random.randint(-10, 10), 100)
    
    # MQL based on score (higher score = higher MQL probability)
    mql_prob = min(0.1 + (lead_score - 40) * 0.015, 0.7)
    mql_date = None
    sql_date = None
    
    if random.random() < mql_prob:
        mql_date = created_date + timedelta(days=random.randint(1, 30))
        if mql_date > END_DATE:
            mql_date = None
        elif random.random() < 0.20:  # 20% MQL to SQL
            sql_date = mql_date + timedelta(days=random.randint(5, 45))
            if sql_date > END_DATE:
                sql_date = None
    
    # Company size (with some missing values)
    if random.random() < 0.1:
        company_size = None
    else:
        size_map = {1: '1000+', 2: '200-999', 3: '1-199'}
        # Sometimes mismatch between lead-provided and account data
        if random.random() < 0.85:
            company_size = size_map[account['tier']]
        else:
            company_size = random.choice(['1-199', '200-999', '1000+'])
    
    lead_id = f'LEAD-{i+1:05d}'
    
    leads.append({
        'lead_id': lead_id,
        'created_date': created_date,
        'email': generate_email(full_name, account['company_name']),
        'company_name': company_name,
        'company_size': company_size,
        'industry': account['industry'] if random.random() < 0.9 else random.choice(INDUSTRIES),
        'lead_source': lead_source,
        'self_reported_source': self_reported,
        'lead_score': lead_score,
        'mql_date': mql_date,
        'sql_date': sql_date
    })
    
    contact_to_account.append({
        'contact_id': lead_id,
        'account_id': account['account_id']
    })

# Add some duplicate leads (same person, slightly different data)
num_duplicates = int(NUM_LEADS * 0.03)  # 3% duplicates
for _ in range(num_duplicates):
    original = random.choice(leads)
    duplicate = original.copy()
    duplicate['lead_id'] = f'LEAD-{len(leads)+1:05d}'
    duplicate['created_date'] = original['created_date'] + timedelta(days=random.randint(1, 60))
    # Slight email variation
    if random.random() < 0.5:
        duplicate['email'] = duplicate['email'].replace('.', '_', 1)
    leads.append(duplicate)
    
    # Don't add to contact_to_account (simulating the data issue)

leads_df = pd.DataFrame(leads)
contact_to_account_df = pd.DataFrame(contact_to_account)

print(f"Generated {len(leads_df)} leads (including {num_duplicates} duplicates)")
print(f"MQL count: {leads_df['mql_date'].notna().sum()}")
print(f"SQL count: {leads_df['sql_date'].notna().sum()}")

In [ ]:
# Generate Touches (marketing touchpoints)
CHANNELS = ['paid_search', 'linkedin', 'organic', 'email', 'webinar', 'content', 'direct', 'retargeting']
CAMPAIGNS = {
    'paid_search': ['Brand Search', 'Competitor Search', 'Product Search', 'Generic Search'],
    'linkedin': ['LinkedIn Sponsored Content', 'LinkedIn InMail', 'LinkedIn Video', 'LinkedIn Carousel'],
    'organic': ['Blog - SEO', 'Documentation', 'Resource Center'],
    'email': ['Newsletter', 'Nurture Sequence', 'Product Update', 'Event Invite', 'Re-engagement'],
    'webinar': ['Monthly Webinar', 'Product Demo Webinar', 'Industry Expert Webinar'],
    'content': ['Ebook Download', 'Whitepaper', 'Case Study', 'ROI Calculator', 'Template Download'],
    'direct': ['Direct Visit', 'Typed URL'],
    'retargeting': ['Display Retargeting', 'LinkedIn Retargeting', 'Google Retargeting']
}
CONTENT_ASSETS = {
    'ebook': ['Complete Guide to B2B Marketing', 'Data-Driven Marketing Playbook', 'ABM Strategy Guide'],
    'whitepaper': ['State of B2B Marketing 2024', 'Attribution Modeling Best Practices', 'Martech Stack Optimization'],
    'case_study': ['Case Study: TechCorp', 'Case Study: DataFlow Inc', 'Case Study: CloudFirst'],
    'webinar': ['Webinar: Pipeline Acceleration', 'Webinar: Attribution Deep Dive', 'Webinar: ABM Tactics'],
    'blog': ['Blog: 10 Tips for B2B Success', 'Blog: Understanding MQL', 'Blog: Sales Marketing Alignment']
}

touches = []
touch_id = 1

for _, lead in leads_df.iterrows():
    # Determine number of touches based on lead progression
    if lead['sql_date'] is not None:
        num_touches = random.randint(8, 20)  # Converted leads have more touches
    elif lead['mql_date'] is not None:
        num_touches = random.randint(4, 12)
    else:
        num_touches = random.randint(1, 5)
    
    # First touch is typically the lead source
    first_touch_channel = lead['lead_source']
    touch_date = lead['created_date']
    
    for i in range(num_touches):
        if i == 0:
            channel = first_touch_channel
        else:
            # Subsequent touches follow realistic patterns
            channel_weights = {
                'email': 0.30,
                'organic': 0.15,
                'content': 0.15,
                'retargeting': 0.15,
                'direct': 0.10,
                'linkedin': 0.08,
                'webinar': 0.05,
                'paid_search': 0.02
            }
            channel = random.choices(list(channel_weights.keys()), 
                                    weights=list(channel_weights.values()))[0]
        
        campaign = random.choice(CAMPAIGNS.get(channel, ['Unknown Campaign']))
        
        # Content asset (sometimes present)
        if channel in ['content', 'organic', 'email', 'webinar']:
            asset_type = random.choice(list(CONTENT_ASSETS.keys()))
            content_asset = random.choice(CONTENT_ASSETS[asset_type])
        else:
            content_asset = None
        
        touches.append({
            'touch_id': f'TOUCH-{touch_id:06d}',
            'lead_id': lead['lead_id'],
            'timestamp': touch_date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59)),
            'channel': channel,
            'campaign': campaign,
            'content_asset': content_asset
        })
        touch_id += 1
        
        # Next touch is 1-14 days later
        touch_date = touch_date + timedelta(days=random.randint(1, 14))
        if touch_date > END_DATE:
            break

touches_df = pd.DataFrame(touches)
print(f"Generated {len(touches_df)} touchpoints")
print(f"Average touches per lead: {len(touches_df) / len(leads_df):.1f}")

In [ ]:
# Generate Opportunities
opportunities = []
opp_id = 1

# Get SQL leads to create opportunities from
sql_leads = leads_df[leads_df['sql_date'].notna()].copy()

# 25% of SQLs become opportunities
opp_leads = sql_leads.sample(frac=0.25)

for _, lead in opp_leads.iterrows():
    # Get the account
    contact_mapping = contact_to_account_df[contact_to_account_df['contact_id'] == lead['lead_id']]
    if len(contact_mapping) == 0:
        continue
    account_id = contact_mapping.iloc[0]['account_id']
    account = accounts_df[accounts_df['account_id'] == account_id].iloc[0]
    
    created_date = lead['sql_date'] + timedelta(days=random.randint(1, 14))
    if created_date > END_DATE:
        continue
    
    # Deal size based on company size
    if account['tier'] == 1:
        base_amount = random.randint(50000, 250000)
    elif account['tier'] == 2:
        base_amount = random.randint(20000, 80000)
    else:
        base_amount = random.randint(5000, 30000)
    
    # Win rate ~30%
    is_won = random.random() < 0.30
    
    # Sales cycle 60-120 days
    cycle_length = random.randint(60, 120)
    close_date = created_date + timedelta(days=cycle_length)
    
    if close_date > END_DATE:
        # Still open
        stage = random.choice(['Discovery', 'Qualification', 'Proposal', 'Negotiation'])
        close_date = None
        is_won = None
    else:
        stage = 'Closed Won' if is_won else 'Closed Lost'
    
    opportunities.append({
        'opp_id': f'OPP-{opp_id:04d}',
        'account_id': account_id,
        'created_date': created_date,
        'close_date': close_date,
        'stage': stage,
        'amount': base_amount,
        'is_won': is_won,
        'primary_lead_id': lead['lead_id']
    })
    opp_id += 1

opportunities_df = pd.DataFrame(opportunities)
print(f"Generated {len(opportunities_df)} opportunities")
print(f"Won: {(opportunities_df['is_won'] == True).sum()}")
print(f"Lost: {(opportunities_df['is_won'] == False).sum()}")
print(f"Open: {opportunities_df['is_won'].isna().sum()}")

In [ ]:
# Summary of generated data
print("=" * 50)
print("DATASET SUMMARY")
print("=" * 50)
print(f"\nAccounts: {len(accounts_df)}")
print(f"Leads: {len(leads_df)}")
print(f"Touches: {len(touches_df)}")
print(f"Opportunities: {len(opportunities_df)}")
print(f"Contact-Account Mappings: {len(contact_to_account_df)}")

print("\n--- Data Quality Issues (Intentional) ---")
print(f"Leads with missing company_size: {leads_df['company_size'].isna().sum()}")
print(f"Leads with missing self_reported_source: {leads_df['self_reported_source'].isna().sum()}")
print(f"Duplicate leads (approx): {num_duplicates}")

---
## Exercise 1: Data Cleaning

Real B2B data is messy. Before any analysis, you need to clean it.

**Tasks:**
1. Find and remove duplicate leads
2. Standardize company names
3. Handle missing values
4. Normalize lead sources

In [ ]:
# Exercise 1.1: Find duplicate leads
# Duplicates might have same email (with slight variations) or same name + company

# Create a working copy
leads_clean = leads_df.copy()

# Normalize email for comparison
leads_clean['email_normalized'] = leads_clean['email'].str.lower().str.replace('_', '.')

# Find duplicates based on normalized email
duplicates = leads_clean[leads_clean.duplicated(subset=['email_normalized'], keep=False)]
print(f"Found {len(duplicates)} records that share an email with another record")
print(f"\nExample duplicates:")
duplicates.head(10)[['lead_id', 'email', 'email_normalized', 'created_date', 'company_name']]

In [ ]:
# Exercise 1.1 Solution: Remove duplicates, keeping the earliest lead
leads_clean = leads_clean.sort_values('created_date')
leads_clean = leads_clean.drop_duplicates(subset=['email_normalized'], keep='first')
print(f"After deduplication: {len(leads_clean)} leads (removed {len(leads_df) - len(leads_clean)})")

In [ ]:
# Exercise 1.2: Standardize company names
# Check the variations
print("Sample of company name variations:")
leads_clean['company_name'].value_counts().head(20)

In [ ]:
# Exercise 1.2 Solution: Standardize company names
def standardize_company_name(name):
    if pd.isna(name):
        return name
    # Convert to title case
    name = name.strip()
    # Remove common suffixes for matching
    suffixes_to_remove = [', Inc.', ', Inc', ' Inc.', ' Inc', ' LLC', ' Corp', ' Corp.', ' Co.', ' Co']
    name_clean = name
    for suffix in suffixes_to_remove:
        name_clean = name_clean.replace(suffix, '')
    # Title case
    name_clean = name_clean.title()
    return name_clean

leads_clean['company_name_std'] = leads_clean['company_name'].apply(standardize_company_name)

print("Unique company names before standardization:", leads_clean['company_name'].nunique())
print("Unique company names after standardization:", leads_clean['company_name_std'].nunique())

In [ ]:
# Exercise 1.3: Handle missing values
print("Missing values per column:")
print(leads_clean.isnull().sum())
print("\n" + "="*40)

# For company_size: we could impute based on account data or leave as 'Unknown'
leads_clean['company_size'] = leads_clean['company_size'].fillna('Unknown')

# For self_reported_source: leave as NaN - it's meaningful (person didn't answer)
print("\nAfter handling missing values:")
print(leads_clean.isnull().sum())

In [ ]:
# Exercise 1.4: Normalize lead sources (already clean in our data, but demonstrate the pattern)
print("Lead source distribution:")
print(leads_clean['lead_source'].value_counts())

# Example: if we had variations like 'LinkedIn', 'linkedin', 'LINKEDIN'
leads_clean['lead_source'] = leads_clean['lead_source'].str.lower().str.strip()

---
## Exercise 2: SQL Queries

Real-world marketing data typically lives in databases (data warehouses, CRMs, etc.), so you need SQL skills. We'll use SQLite to run actual SQL queries against our synthetic data.

**Tasks:**
1. Conversion rates by stage
2. Average deal size by segment
3. Lead volume by source over time
4. Pipeline velocity calculation

In [ ]:
# Set up SQLite in-memory database
import sqlite3

# Create connection and load data
conn = sqlite3.connect(':memory:')

# Load all dataframes into SQL tables
leads_clean.to_sql('leads', conn, index=False, if_exists='replace')
accounts_df.to_sql('accounts', conn, index=False, if_exists='replace')
touches_df.to_sql('touches', conn, index=False, if_exists='replace')
opportunities_df.to_sql('opportunities', conn, index=False, if_exists='replace')
contact_to_account_df.to_sql('contact_to_account', conn, index=False, if_exists='replace')

# Helper function to run SQL and display results
def run_sql(query, conn=conn):
    """Execute SQL query and return results as DataFrame."""
    return pd.read_sql_query(query, conn)

print("Database tables created:")
print(run_sql("SELECT name FROM sqlite_master WHERE type='table'"))

In [ ]:
# Exercise 2.1: Conversion rates by stage

query = """
SELECT 
    COUNT(*) as total_leads,
    SUM(CASE WHEN mql_date IS NOT NULL THEN 1 ELSE 0 END) as mqls,
    SUM(CASE WHEN sql_date IS NOT NULL THEN 1 ELSE 0 END) as sqls,
    ROUND(SUM(CASE WHEN mql_date IS NOT NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as lead_to_mql_pct,
    ROUND(SUM(CASE WHEN sql_date IS NOT NULL THEN 1 ELSE 0 END) * 100.0 / 
          NULLIF(SUM(CASE WHEN mql_date IS NOT NULL THEN 1 ELSE 0 END), 0), 2) as mql_to_sql_pct
FROM leads;
"""

print("Funnel Conversion Rates (SQL)")
print("=" * 50)
result = run_sql(query)
print(result.to_string(index=False))

# Add opportunity conversions
opp_query = """
SELECT 
    COUNT(*) as total_opportunities,
    SUM(CASE WHEN is_won = 1 THEN 1 ELSE 0 END) as closed_won,
    ROUND(SUM(CASE WHEN is_won = 1 THEN 1 ELSE 0 END) * 100.0 / 
          NULLIF(SUM(CASE WHEN is_won IS NOT NULL THEN 1 ELSE 0 END), 0), 2) as win_rate_pct
FROM opportunities;
"""
print("\nOpportunity Win Rate:")
print(run_sql(opp_query).to_string(index=False))

In [ ]:
# Exercise 2.2: Average deal size by segment

query = """
SELECT 
    a.tier,
    COUNT(o.opp_id) as num_opps,
    ROUND(AVG(o.amount), 0) as avg_deal_size,
    SUM(o.amount) as total_pipeline
FROM opportunities o
JOIN accounts a ON o.account_id = a.account_id
GROUP BY a.tier
ORDER BY a.tier;
"""

print("Deal Size by Account Tier (SQL)")
print("=" * 50)
print(run_sql(query).to_string(index=False))

# By industry
query_industry = """
SELECT 
    a.industry,
    COUNT(o.opp_id) as num_opps,
    ROUND(AVG(o.amount), 0) as avg_deal_size,
    SUM(o.amount) as total_pipeline
FROM opportunities o
JOIN accounts a ON o.account_id = a.account_id
GROUP BY a.industry
ORDER BY avg_deal_size DESC;
"""

print("\nDeal Size by Industry (SQL)")
print("=" * 50)
print(run_sql(query_industry).to_string(index=False))

In [ ]:
# Exercise 2.3: Lead volume by source over time
# Note: SQLite uses strftime instead of DATE_TRUNC

query = """
SELECT 
    strftime('%Y-%m', created_date) as month,
    lead_source,
    COUNT(*) as lead_count
FROM leads
GROUP BY strftime('%Y-%m', created_date), lead_source
ORDER BY month, lead_source;
"""

result = run_sql(query)

# Pivot for better readability
pivot = result.pivot(index='month', columns='lead_source', values='lead_count').fillna(0).astype(int)
print("Monthly Lead Volume by Source (SQL)")
print("=" * 60)
print(pivot)

In [ ]:
# Exercise 2.4: Pipeline velocity calculation
# Velocity = (Number of Opps × Win Rate × Avg Deal Size) / Sales Cycle Length

query = """
WITH closed_deals AS (
    SELECT 
        COUNT(*) as total_closed,
        SUM(CASE WHEN is_won = 1 THEN 1 ELSE 0 END) as won_opps,
        AVG(amount) as avg_deal_size,
        AVG(julianday(close_date) - julianday(created_date)) as avg_cycle_days
    FROM opportunities
    WHERE close_date IS NOT NULL
)
SELECT 
    total_closed,
    won_opps,
    ROUND(won_opps * 100.0 / total_closed, 1) as win_rate_pct,
    ROUND(avg_deal_size, 0) as avg_deal_size,
    ROUND(avg_cycle_days, 0) as avg_cycle_days,
    ROUND((won_opps * avg_deal_size) / avg_cycle_days, 0) as daily_velocity
FROM closed_deals;
"""

print("Pipeline Velocity Analysis (SQL)")
print("=" * 50)
result = run_sql(query)
print(result.to_string(index=False))

# Calculate monthly velocity
daily_vel = result['daily_velocity'].values[0]
print(f"\nMonthly Revenue Velocity: ${daily_vel * 30:,.0f}/month")

In [ ]:
# Store key metrics as variables for later exercises
# (These are used in visualization exercises later)

total_leads = len(leads_clean)
total_mqls = leads_clean['mql_date'].notna().sum()
total_sqls = leads_clean['sql_date'].notna().sum()
total_opps = len(opportunities_df)
won_opps = (opportunities_df['is_won'] == True).sum()

print("Summary metrics stored for later exercises:")
print(f"  total_leads: {total_leads:,}")
print(f"  total_mqls: {total_mqls:,}")
print(f"  total_sqls: {total_sqls:,}")
print(f"  total_opps: {total_opps:,}")
print(f"  won_opps: {won_opps:,}")

---
## Exercise 3: Conversion Rate Analysis

Dig deeper into conversion rates by segment to identify best and worst performers.

**Tasks:**
1. Calculate stage-by-stage conversion rates
2. Segment by lead source, company size, industry
3. Identify best/worst performing segments

In [ ]:
# Exercise 3.1: Stage-by-stage conversion rates by lead source

def calculate_conversion_rates(df, group_col):
    """Calculate funnel conversion rates grouped by a column."""
    grouped = df.groupby(group_col).agg({
        'lead_id': 'count',
        'mql_date': lambda x: x.notna().sum(),
        'sql_date': lambda x: x.notna().sum()
    })
    grouped.columns = ['leads', 'mqls', 'sqls']
    
    grouped['lead_to_mql'] = (grouped['mqls'] / grouped['leads'] * 100).round(1)
    grouped['mql_to_sql'] = (grouped['sqls'] / grouped['mqls'] * 100).round(1)
    
    return grouped

# By lead source
source_conversion = calculate_conversion_rates(leads_clean, 'lead_source')
print("Conversion Rates by Lead Source")
print(source_conversion.sort_values('lead_to_mql', ascending=False))

In [ ]:
# Exercise 3.2: By company size
size_conversion = calculate_conversion_rates(leads_clean, 'company_size')
print("Conversion Rates by Company Size")
print(size_conversion.sort_values('lead_to_mql', ascending=False))

In [ ]:
# Exercise 3.3: By industry
industry_conversion = calculate_conversion_rates(leads_clean, 'industry')
print("Conversion Rates by Industry")
print(industry_conversion.sort_values('lead_to_mql', ascending=False))

In [ ]:
# Exercise 3.4: Identify best and worst performing segments
# Combine source + company size for deeper analysis

leads_clean['segment'] = leads_clean['lead_source'] + ' | ' + leads_clean['company_size']

segment_conversion = calculate_conversion_rates(leads_clean, 'segment')
# Filter for segments with enough volume (at least 50 leads)
segment_conversion = segment_conversion[segment_conversion['leads'] >= 50]

print("TOP 5 SEGMENTS (by Lead→MQL conversion)")
print(segment_conversion.sort_values('lead_to_mql', ascending=False).head())

print("\n" + "="*50)

print("\nBOTTOM 5 SEGMENTS (by Lead→MQL conversion)")
print(segment_conversion.sort_values('lead_to_mql', ascending=True).head())

---
## Exercise 4: Build Attribution Models

Implement the core attribution models from scratch.

**Tasks:**
1. First-touch attribution
2. Last-touch attribution
3. Linear attribution
4. Time-decay attribution

In [ ]:
# Get touches for leads that became opportunities (where we have pipeline to attribute)
opp_lead_ids = opportunities_df['primary_lead_id'].tolist()
opp_touches = touches_df[touches_df['lead_id'].isin(opp_lead_ids)].copy()

# Merge with opportunity amount for attribution
opp_touches = opp_touches.merge(
    opportunities_df[['primary_lead_id', 'amount', 'opp_id']], 
    left_on='lead_id', 
    right_on='primary_lead_id',
    how='left'
)

print(f"Touchpoints for opportunities: {len(opp_touches)}")
print(f"Unique opportunities: {opp_touches['opp_id'].nunique()}")

In [ ]:
# Exercise 4.1: First-Touch Attribution
# All credit goes to the first touchpoint

def first_touch_attribution(touches_df):
    """Attribute all pipeline credit to the first touch."""
    # Sort by timestamp and get first touch per opportunity
    first_touches = (touches_df
                     .sort_values('timestamp')
                     .groupby('opp_id')
                     .first()
                     .reset_index())
    
    # Sum by channel
    attribution = (first_touches
                   .groupby('channel')
                   .agg({'amount': 'sum', 'opp_id': 'count'})
                   .rename(columns={'amount': 'pipeline', 'opp_id': 'opps'}))
    
    return attribution.sort_values('pipeline', ascending=False)

first_touch = first_touch_attribution(opp_touches)
print("First-Touch Attribution")
print(first_touch)

In [ ]:
# Exercise 4.2: Last-Touch Attribution
# All credit goes to the last touchpoint before opportunity creation

def last_touch_attribution(touches_df):
    """Attribute all pipeline credit to the last touch."""
    # Sort by timestamp and get last touch per opportunity
    last_touches = (touches_df
                    .sort_values('timestamp')
                    .groupby('opp_id')
                    .last()
                    .reset_index())
    
    # Sum by channel
    attribution = (last_touches
                   .groupby('channel')
                   .agg({'amount': 'sum', 'opp_id': 'count'})
                   .rename(columns={'amount': 'pipeline', 'opp_id': 'opps'}))
    
    return attribution.sort_values('pipeline', ascending=False)

last_touch = last_touch_attribution(opp_touches)
print("Last-Touch Attribution")
print(last_touch)

In [ ]:
# Exercise 4.3: Linear Attribution
# Credit split equally across all touchpoints

def linear_attribution(touches_df):
    """Split pipeline credit equally across all touches."""
    # Count touches per opportunity
    touch_counts = touches_df.groupby('opp_id').size().reset_index(name='touch_count')
    
    # Merge back to get credit per touch
    touches_with_counts = touches_df.merge(touch_counts, on='opp_id')
    touches_with_counts['credit'] = touches_with_counts['amount'] / touches_with_counts['touch_count']
    
    # Sum by channel
    attribution = (touches_with_counts
                   .groupby('channel')
                   .agg({'credit': 'sum', 'touch_id': 'count'})
                   .rename(columns={'credit': 'pipeline', 'touch_id': 'touches'}))
    
    return attribution.sort_values('pipeline', ascending=False)

linear = linear_attribution(opp_touches)
print("Linear Attribution")
print(linear)

In [ ]:
# Exercise 4.4: Time-Decay Attribution
# More recent touches get more credit (exponential decay)

def time_decay_attribution(touches_df, half_life_days=7):
    """Attribute more credit to recent touches using exponential decay."""
    df = touches_df.copy()
    
    # Get the last touch timestamp per opportunity
    last_touch_time = df.groupby('opp_id')['timestamp'].max().reset_index()
    last_touch_time.columns = ['opp_id', 'last_touch']
    
    df = df.merge(last_touch_time, on='opp_id')
    
    # Calculate days before last touch
    df['days_before_last'] = (pd.to_datetime(df['last_touch']) - 
                               pd.to_datetime(df['timestamp'])).dt.days
    
    # Apply exponential decay: weight = 2^(-days/half_life)
    df['weight'] = 2 ** (-df['days_before_last'] / half_life_days)
    
    # Normalize weights within each opportunity
    weight_sums = df.groupby('opp_id')['weight'].sum().reset_index(name='weight_sum')
    df = df.merge(weight_sums, on='opp_id')
    df['normalized_weight'] = df['weight'] / df['weight_sum']
    
    # Calculate credit
    df['credit'] = df['amount'] * df['normalized_weight']
    
    # Sum by channel
    attribution = (df
                   .groupby('channel')
                   .agg({'credit': 'sum', 'touch_id': 'count'})
                   .rename(columns={'credit': 'pipeline', 'touch_id': 'touches'}))
    
    return attribution.sort_values('pipeline', ascending=False)

time_decay = time_decay_attribution(opp_touches, half_life_days=7)
print("Time-Decay Attribution (7-day half-life)")
print(time_decay)

---
## Exercise 5: Attribution Comparison Analysis

Compare how different attribution models value each channel.

**Tasks:**
1. Compare channel rankings across models
2. Visualize differences
3. Write up implications for budget allocation

In [ ]:
# Exercise 5.1: Compare channel rankings

# Combine all models
comparison = pd.DataFrame({
    'first_touch': first_touch['pipeline'],
    'last_touch': last_touch['pipeline'],
    'linear': linear['pipeline'],
    'time_decay': time_decay['pipeline']
}).fillna(0)

# Add percentage of total
for col in comparison.columns:
    comparison[f'{col}_pct'] = (comparison[col] / comparison[col].sum() * 100).round(1)

print("Attribution Comparison (Pipeline $)")
print(comparison[['first_touch', 'last_touch', 'linear', 'time_decay']].round(0))

In [ ]:
# Exercise 5.2: Visualize differences
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))

x = range(len(comparison))
width = 0.2

bars1 = ax.bar([i - 1.5*width for i in x], comparison['first_touch']/1000, width, label='First Touch', color='#2563eb')
bars2 = ax.bar([i - 0.5*width for i in x], comparison['last_touch']/1000, width, label='Last Touch', color='#7c3aed')
bars3 = ax.bar([i + 0.5*width for i in x], comparison['linear']/1000, width, label='Linear', color='#059669')
bars4 = ax.bar([i + 1.5*width for i in x], comparison['time_decay']/1000, width, label='Time Decay', color='#dc2626')

ax.set_ylabel('Pipeline ($K)')
ax.set_title('Attribution Model Comparison by Channel')
ax.set_xticks(x)
ax.set_xticklabels(comparison.index, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('attribution_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Exercise 5.3: Identify winners and losers by model

# Calculate rank for each channel under each model
rankings = comparison[['first_touch', 'last_touch', 'linear', 'time_decay']].rank(ascending=False)
rankings.columns = ['ft_rank', 'lt_rank', 'lin_rank', 'td_rank']

# Find channels with biggest rank changes
rankings['rank_variance'] = rankings.var(axis=1)
rankings = rankings.sort_values('rank_variance', ascending=False)

print("Channel Rankings by Attribution Model")
print("(1 = highest pipeline credit)")
print(rankings)

### Attribution Implications for Budget Allocation

**Key Observations:**

1. **First-touch favors awareness channels** (paid search, LinkedIn, content syndication) that bring people in initially. If you use first-touch, you'll invest more in top-of-funnel.

2. **Last-touch favors conversion channels** (direct, retargeting, email) that catch people right before they convert. This often overstates the value of branded search and remarketing.

3. **Linear gives a balanced view** but can undervalue both introduction and closing channels by spreading credit too thin.

4. **Time-decay is often most realistic for B2B** because it recognizes that recent touches matter more, while still giving credit to earlier touches.

**Recommendation:** Don't rely on a single model. Present multiple models to stakeholders and explain the trade-offs. Time-decay is often a good default for B2B.

---
## Exercise 6: Self-Reported vs Tracked Attribution

Compare what people say ("how did you hear about us?") vs what tracking shows.

**Tasks:**
1. Compare self-reported source to first-touch tracked source
2. Calculate agreement rate
3. Identify dark funnel indicators

In [ ]:
# Exercise 6.1: Map self-reported sources to standard categories

def categorize_self_reported(text):
    """Map free-text self-reported sources to standard categories."""
    if pd.isna(text):
        return 'not_provided'
    
    text = text.lower()
    
    if any(x in text for x in ['google', 'search', 'seo', 'found your website']):
        return 'search'
    elif any(x in text for x in ['linkedin', 'li']):
        return 'linkedin'
    elif any(x in text for x in ['referral', 'colleague', 'friend', 'recommend', 'word of mouth', 'boss', 'peer']):
        return 'referral'
    elif any(x in text for x in ['webinar', 'conference', 'event', 'trade show', 'dreamforce', 'saastr']):
        return 'event'
    elif any(x in text for x in ['blog', 'article', 'content', 'whitepaper', 'ebook', 'report', 'download']):
        return 'content'
    elif any(x in text for x in ['podcast', 'youtube', 'twitter', 'slack', 'newsletter', 'following']):
        return 'dark_funnel'  # Channels we can't track well
    elif any(x in text for x in ["can't remember", 'not sure']):
        return 'unknown'
    else:
        return 'other'

leads_clean['self_reported_category'] = leads_clean['self_reported_source'].apply(categorize_self_reported)

print("Self-Reported Source Categories")
print(leads_clean['self_reported_category'].value_counts())

In [ ]:
# Exercise 6.2: Get first-touch for each lead

first_touches_all = (touches_df
                     .sort_values('timestamp')
                     .groupby('lead_id')
                     .first()
                     .reset_index()[['lead_id', 'channel']])

first_touches_all.columns = ['lead_id', 'first_touch_channel']

# Merge with leads
leads_with_ft = leads_clean.merge(first_touches_all, on='lead_id', how='left')

# Map first-touch to comparable categories
def map_tracked_to_category(channel):
    if pd.isna(channel):
        return 'unknown'
    
    mapping = {
        'paid_search': 'search',
        'organic': 'search',
        'linkedin': 'linkedin',
        'referral': 'referral',
        'event': 'event',
        'webinar': 'event',
        'content': 'content',
        'content_syndication': 'content',
        'email': 'email',
        'direct': 'direct',
        'retargeting': 'retargeting'
    }
    return mapping.get(channel, 'other')

leads_with_ft['first_touch_category'] = leads_with_ft['first_touch_channel'].apply(map_tracked_to_category)

In [ ]:
# Exercise 6.3: Calculate agreement rate

# Exclude cases where self-reported wasn't provided
comparable = leads_with_ft[
    (leads_with_ft['self_reported_category'] != 'not_provided') &
    (leads_with_ft['self_reported_category'] != 'unknown')
].copy()

comparable['match'] = comparable['self_reported_category'] == comparable['first_touch_category']

agreement_rate = comparable['match'].mean()
print(f"Agreement Rate: {agreement_rate:.1%}")
print(f"(Self-reported matches first-touch tracking)")

In [ ]:
# Exercise 6.4: Cross-tabulation to see patterns

crosstab = pd.crosstab(
    comparable['first_touch_category'], 
    comparable['self_reported_category'],
    margins=True
)

print("Cross-tab: First Touch (rows) vs Self-Reported (columns)")
print(crosstab)

In [ ]:
# Exercise 6.5: Dark funnel analysis

dark_funnel_leads = leads_with_ft[leads_with_ft['self_reported_category'] == 'dark_funnel']

print(f"Dark Funnel Leads: {len(dark_funnel_leads)} ({100*len(dark_funnel_leads)/len(leads_with_ft):.1f}% of total)")
print("\nThese leads self-reported sources we can't track (podcasts, YouTube, community, etc.)")
print("\nWhat tracking THOUGHT was their first touch:")
print(dark_funnel_leads['first_touch_channel'].value_counts())

### Dark Funnel Insights

The dark funnel represents touchpoints we can't track:
- Word of mouth
- Podcasts
- Private communities (Slack, Discord)
- YouTube
- Social (organic, not ads)

**Key insight:** When someone says they heard about us on a podcast but tracking shows "organic search" as first touch, the tracking is wrong. They searched for us *because* of the podcast.

**Recommendations:**
1. Always collect "how did you hear about us?" - it catches what tracking misses
2. Don't treat first-touch tracking as truth - it's often the first *trackable* touch
3. Invest in dark funnel channels (podcasts, community) even if they're hard to attribute

---
## Exercise 7: Cohort Analysis

Track how cohorts of leads progress through the funnel over time.

**Tasks:**
1. Define cohorts by signup month
2. Track conversion rates over time
3. Create cohort heatmap

In [ ]:
# Exercise 7.1: Define cohorts

leads_clean['cohort'] = pd.to_datetime(leads_clean['created_date']).dt.to_period('M')

# Calculate days to MQL for each lead
leads_clean['days_to_mql'] = (
    pd.to_datetime(leads_clean['mql_date']) - 
    pd.to_datetime(leads_clean['created_date'])
).dt.days

print("Leads per cohort:")
print(leads_clean['cohort'].value_counts().sort_index())

In [ ]:
# Exercise 7.2: Track conversion at different time intervals

def cohort_conversion_by_days(leads_df, max_days_list=[7, 14, 30, 60, 90]):
    """Calculate MQL conversion rate at different days after signup."""
    cohorts = leads_df.groupby('cohort').size().to_frame('total_leads')
    
    for days in max_days_list:
        # Count leads that converted within X days
        converted = leads_df[
            (leads_df['days_to_mql'].notna()) & 
            (leads_df['days_to_mql'] <= days)
        ].groupby('cohort').size()
        
        cohorts[f'mql_{days}d'] = converted
        cohorts[f'mql_{days}d_pct'] = (cohorts[f'mql_{days}d'] / cohorts['total_leads'] * 100).round(1)
    
    return cohorts.fillna(0)

cohort_analysis = cohort_conversion_by_days(leads_clean)
print("Cohort Analysis: MQL Conversion by Days After Signup")
print(cohort_analysis[['total_leads', 'mql_7d_pct', 'mql_14d_pct', 'mql_30d_pct', 'mql_60d_pct', 'mql_90d_pct']])

In [ ]:
# Exercise 7.3: Create cohort heatmap

# Prepare data for heatmap
heatmap_data = cohort_analysis[['mql_7d_pct', 'mql_14d_pct', 'mql_30d_pct', 'mql_60d_pct', 'mql_90d_pct']].copy()
heatmap_data.columns = ['7 days', '14 days', '30 days', '60 days', '90 days']

# Only show cohorts with enough time to mature (exclude last 3 months for 90-day metric)
heatmap_data = heatmap_data.iloc[:-3]

fig, ax = plt.subplots(figsize=(10, 8))

im = ax.imshow(heatmap_data.values, cmap='YlGn', aspect='auto')

ax.set_xticks(range(len(heatmap_data.columns)))
ax.set_xticklabels(heatmap_data.columns)
ax.set_yticks(range(len(heatmap_data.index)))
ax.set_yticklabels([str(p) for p in heatmap_data.index])

# Add text annotations
for i in range(len(heatmap_data.index)):
    for j in range(len(heatmap_data.columns)):
        text = ax.text(j, i, f'{heatmap_data.iloc[i, j]:.1f}%',
                       ha='center', va='center', color='black', fontsize=9)

ax.set_xlabel('Days Since Signup')
ax.set_ylabel('Cohort (Month)')
ax.set_title('MQL Conversion Rate by Cohort and Time')

plt.colorbar(im, label='Conversion Rate %')
plt.tight_layout()
plt.savefig('cohort_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Exercise 8: Pipeline Contribution Report

Build a marketing pipeline contribution report.

**Tasks:**
1. Calculate sourced pipeline by channel (first-touch)
2. Calculate influenced pipeline by channel (any-touch)
3. Build summary report

In [ ]:
# Exercise 8.1: Sourced pipeline (first-touch)
# Pipeline where the channel was the FIRST touchpoint

sourced = first_touch_attribution(opp_touches)
sourced.columns = ['sourced_pipeline', 'sourced_opps']

print("Sourced Pipeline by Channel")
print(sourced)

In [ ]:
# Exercise 8.2: Influenced pipeline (any-touch)
# Pipeline where the channel touched the deal at ANY point

def influenced_pipeline(touches_df):
    """Calculate pipeline where channel had any touch."""
    # Get unique channel-opportunity combinations
    channel_opps = touches_df[['channel', 'opp_id', 'amount']].drop_duplicates(subset=['channel', 'opp_id'])
    
    # Sum by channel (each opp counted once per channel)
    influenced = channel_opps.groupby('channel').agg({
        'amount': 'sum',
        'opp_id': 'count'
    })
    influenced.columns = ['influenced_pipeline', 'influenced_opps']
    
    return influenced.sort_values('influenced_pipeline', ascending=False)

influenced = influenced_pipeline(opp_touches)
print("Influenced Pipeline by Channel")
print(influenced)

In [ ]:
# Exercise 8.3: Combined report

# Merge sourced and influenced
pipeline_report = sourced.join(influenced, how='outer').fillna(0)

# Add totals
total_pipeline = opportunities_df['amount'].sum()
pipeline_report['sourced_pct'] = (pipeline_report['sourced_pipeline'] / total_pipeline * 100).round(1)
pipeline_report['influenced_pct'] = (pipeline_report['influenced_pipeline'] / total_pipeline * 100).round(1)

# Influenced / Sourced ratio (how much leverage does a channel have?)
pipeline_report['leverage_ratio'] = (pipeline_report['influenced_pipeline'] / 
                                      pipeline_report['sourced_pipeline'].replace(0, 1)).round(2)

print("=" * 60)
print("MARKETING PIPELINE CONTRIBUTION REPORT")
print("=" * 60)
print(f"\nTotal Pipeline: ${total_pipeline:,.0f}")
print(f"Total Opportunities: {len(opportunities_df)}")
print("\n")
print(pipeline_report.sort_values('sourced_pipeline', ascending=False))

In [ ]:
# Visualize sourced vs influenced

fig, ax = plt.subplots(figsize=(12, 6))

x = range(len(pipeline_report))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], pipeline_report['sourced_pipeline']/1000, 
               width, label='Sourced', color='#2563eb')
bars2 = ax.bar([i + width/2 for i in x], pipeline_report['influenced_pipeline']/1000, 
               width, label='Influenced', color='#7c3aed', alpha=0.7)

ax.set_ylabel('Pipeline ($K)')
ax.set_title('Sourced vs Influenced Pipeline by Channel')
ax.set_xticks(x)
ax.set_xticklabels(pipeline_report.index, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('pipeline_contribution.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Exercise 9: Design a Holdout Test

Conceptual exercise: design an incrementality test for LinkedIn ads.

**Scenario:** Marketing wants to know if LinkedIn ads actually drive incremental pipeline, or if those leads would have converted anyway.

### Holdout Test Design

**Objective:** Measure the true incremental impact of LinkedIn advertising on pipeline generation.

**Approach: Geographic Holdout**

1. **Select test and control regions:**
   - Divide your target markets into similar groups (by size, industry mix, historical performance)
   - Example: Hold out 20% of metro areas from LinkedIn ads

2. **Duration:** Run for 90+ days (one full sales cycle)

3. **Measurement:**
   - Track pipeline generated per lead in test vs control regions
   - Control for other marketing activities

4. **Calculate incrementality:**
   - Incremental pipeline = (Test region pipeline/lead - Control region pipeline/lead) × Test leads
   - Lift = (Test - Control) / Control

**Statistical Requirements:**
- Minimum sample size per group to detect 20% lift with 80% power
- Account for multiple comparisons if testing multiple channels

In [ ]:
# Exercise 9: Code scaffold for analyzing holdout test results

def analyze_holdout_test(test_data, control_data, metric='pipeline_per_lead'):
    """
    Analyze results from a holdout/incrementality test.
    
    Parameters:
    - test_data: DataFrame with leads/pipeline in test group (exposed to ads)
    - control_data: DataFrame with leads/pipeline in control group (no ads)
    - metric: What to measure
    
    Returns:
    - Dictionary with test results and statistical significance
    """
    from scipy import stats
    
    # Calculate metrics
    test_metric = test_data[metric].mean()
    control_metric = control_data[metric].mean()
    
    # Lift
    lift = (test_metric - control_metric) / control_metric if control_metric > 0 else 0
    
    # Statistical significance (t-test)
    t_stat, p_value = stats.ttest_ind(test_data[metric], control_data[metric])
    
    # Incremental value
    incremental_per_unit = test_metric - control_metric
    total_incremental = incremental_per_unit * len(test_data)
    
    results = {
        'test_mean': test_metric,
        'control_mean': control_metric,
        'lift': lift,
        'p_value': p_value,
        'is_significant': p_value < 0.05,
        'incremental_per_unit': incremental_per_unit,
        'total_incremental_value': total_incremental
    }
    
    return results

# Example usage (with synthetic data):
np.random.seed(42)
test_group = pd.DataFrame({'pipeline_per_lead': np.random.normal(5000, 2000, 500)})
control_group = pd.DataFrame({'pipeline_per_lead': np.random.normal(4200, 2000, 500)})

results = analyze_holdout_test(test_group, control_group)

print("Holdout Test Results")
print("=" * 40)
print(f"Test Group Mean: ${results['test_mean']:,.0f}")
print(f"Control Group Mean: ${results['control_mean']:,.0f}")
print(f"Lift: {results['lift']:.1%}")
print(f"P-value: {results['p_value']:.4f}")
print(f"Statistically Significant: {results['is_significant']}")
print(f"\nIncremental Pipeline per Lead: ${results['incremental_per_unit']:,.0f}")
print(f"Total Incremental Value: ${results['total_incremental_value']:,.0f}")

---
## Exercise 10: Visualization Exercises

Create the key visualizations every marketing analyst needs.

**Tasks:**
1. Funnel chart
2. Attribution model comparison (done in Ex 5)
3. Cohort heatmap (done in Ex 7)
4. Pipeline waterfall

In [ ]:
# Exercise 10.1: Funnel Chart

funnel_stages = ['Leads', 'MQLs', 'SQLs', 'Opportunities', 'Closed Won']
funnel_values = [total_leads, total_mqls, total_sqls, total_opps, won_opps]

fig, ax = plt.subplots(figsize=(10, 6))

# Create horizontal bar chart (simulating funnel)
colors = ['#3b82f6', '#6366f1', '#8b5cf6', '#a855f7', '#22c55e']
y_positions = range(len(funnel_stages))

# Normalize widths to create funnel effect
max_val = max(funnel_values)
widths = [v for v in funnel_values]

bars = ax.barh(y_positions, widths, color=colors, height=0.7)

# Add labels
for i, (stage, value) in enumerate(zip(funnel_stages, funnel_values)):
    # Conversion rate from previous stage
    if i > 0:
        conv_rate = value / funnel_values[i-1] * 100
        label = f"{stage}: {value:,} ({conv_rate:.1f}%)"
    else:
        label = f"{stage}: {value:,}"
    ax.text(value + max_val*0.02, i, label, va='center', fontsize=11)

ax.set_yticks([])
ax.set_xlim(0, max_val * 1.3)
ax.set_title('Marketing Funnel', fontsize=14, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.savefig('funnel_chart.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Exercise 10.4: Pipeline Waterfall Chart

# Show pipeline changes: Starting → New → Moved Up → Moved Down → Lost → Ending
# Simplified version showing sources of pipeline

pipeline_sources = pipeline_report.sort_values('sourced_pipeline', ascending=False).head(6)

fig, ax = plt.subplots(figsize=(12, 6))

channels = pipeline_sources.index.tolist()
values = pipeline_sources['sourced_pipeline'].values / 1000

# Waterfall logic
cumulative = np.zeros(len(values) + 1)
cumulative[1:] = np.cumsum(values)

# Starting positions for bars
starts = cumulative[:-1]

colors = ['#3b82f6', '#6366f1', '#8b5cf6', '#a855f7', '#ec4899', '#f97316']

bars = ax.bar(channels, values, bottom=starts, color=colors[:len(channels)], edgecolor='white', linewidth=2)

# Add total bar
ax.bar('TOTAL', cumulative[-1], color='#22c55e', edgecolor='white', linewidth=2)

# Add connecting lines
for i in range(len(channels)):
    ax.plot([i + 0.4, i + 0.6], [cumulative[i+1], cumulative[i+1]], 'k-', linewidth=1)

# Labels
for i, (channel, value) in enumerate(zip(channels, values)):
    ax.text(i, starts[i] + value/2, f'${value:.0f}K', ha='center', va='center', 
            fontsize=10, fontweight='bold', color='white')

ax.text(len(channels), cumulative[-1]/2, f'${cumulative[-1]:.0f}K', ha='center', va='center',
        fontsize=12, fontweight='bold', color='white')

ax.set_ylabel('Pipeline ($K)')
ax.set_title('Pipeline Waterfall by Source Channel', fontsize=14, fontweight='bold')
ax.set_xticklabels(channels + ['TOTAL'], rotation=45, ha='right')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('pipeline_waterfall.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Summary

You've now practiced:

1. **Data Cleaning** - Deduplication, standardization, handling missing values
2. **SQL Queries** - Conversion rates, deal sizes, pipeline velocity
3. **Conversion Analysis** - Segmentation, identifying best/worst performers
4. **Attribution Models** - First-touch, last-touch, linear, time-decay
5. **Attribution Comparison** - Understanding model trade-offs
6. **Dark Funnel Analysis** - Self-reported vs tracked attribution
7. **Cohort Analysis** - Tracking conversion over time
8. **Pipeline Reporting** - Sourced vs influenced pipeline
9. **Incrementality Testing** - Holdout test design
10. **Visualization** - Funnels, comparisons, waterfalls

### Next Steps

- Apply these techniques to your actual company data
- Build automated reports using these patterns
- Experiment with different attribution models to see which tells the most useful story for your business

In [ ]:
# Save all dataframes for future use
leads_clean.to_csv('leads_clean.csv', index=False)
accounts_df.to_csv('accounts.csv', index=False)
touches_df.to_csv('touches.csv', index=False)
opportunities_df.to_csv('opportunities.csv', index=False)
contact_to_account_df.to_csv('contact_to_account.csv', index=False)

print("Data saved to CSV files.")